# RNN Based molucule generation 

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery. 

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset. 

There exists a lot of models : 

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models 


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model 


A language model is a model which predict the next token of a sequence given the previous ones : 

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network 

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data. 



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model 

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [1]:
!git clone https://github.com/joeymach/Leveraging-VAE-to-generate-molecules.git ./data/

fatal: destination path './data' already exists and is not an empty directory.


Import pandas and load the first 1000 lines

In [2]:
import pandas as pd

# Import the first 1000 lines
df = pd.read_csv('./data/250k_smiles.csv')

Display the first rows of the dataframe

In [3]:
df.head(1)

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.0506,0.702012,2.084095


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [4]:
df['smiles'].str.len().max()

110


Code a function **unic_characters(string)** which return the unic characters in a string


In [5]:
def unic_characters(string):
    rtn = set()
    
    for char in string:
        rtn.add(char)
        
    return rtn

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [6]:
unic_chars = unic_characters(" ".join(df['smiles']))
nb_chars = len(unic_chars) + 1
len(unic_chars)

36

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [7]:
ending_tocken = "\n"

def map_char_to_int(unic_chars):
    rtn = dict()
    
    for i, char in enumerate(unic_chars):
        rtn[char] = i
        
    rtn[ending_tocken] = len(unic_chars)
    return rtn

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [8]:
def map_int_to_char(unic_chars):
    rtn = dict()
    
    for i, char in enumerate(unic_chars):
        rtn[i] = char
        len(unic_chars)
        
    rtn[len(unic_chars)] = ending_tocken
    return rtn

For each smile molecule add the ending token to it

In [9]:
df["smiles"] = df["smiles"].apply(lambda x: x + ending_tocken)

In [10]:
dict_char_to_int = map_char_to_int(unic_chars)
dict_int_to_char = map_int_to_char(unic_chars)

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

Ca sera 3 car on a 3 autres colonnes

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [11]:
(n_examples, seq_length, n_features) = (len(df),5,1)

def build_X_and_y(string, i_char, seq_length):
    if(i_char >= len(string) or i_char + seq_length >= len(string)):
        return "", ""
    
    X = string[i_char:i_char + seq_length]
    y = string[i_char + seq_length]
    return X, y

for i in range(1, 4):
    print(build_X_and_y("azertyuiop", i, 4))

('zert', 'y')
('erty', 'u')
('rtyu', 'i')


In [12]:
build_X_and_y("azertyuiop", 0, seq_length)

('azert', 'y')

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

In [13]:
X_train = []
y_train = []

for i in df['smiles'].tolist():
    
    X, y = build_X_and_y(i, 0, seq_length)
    
    X_ = []

    y_ = [0] * nb_chars
    y_[dict_char_to_int[y]] = 1
    
    for e in X:
        X_.append(dict_char_to_int[e])
    
    X_train.append(X_)
    y_train.append(y_)
    
X_train[0]

[13, 13, 34, 13, 33]

Create numpy arrays from the lists

In [14]:
import numpy as np
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape

(249455, 5)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [15]:
X_train = X_train.reshape(n_examples, seq_length, 1)

Normalize X by dividing each values by the total number of unic characters

In [16]:
X_train = X_train /len(unic_chars)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

def create_model():

  lstm_neurons = 128

  dropout_rate = 0.2

  dense_neurons = 36

  dense_activation = 'softmax'

  model = Sequential()

  model.add(LSTM(lstm_neurons, input_shape=(seq_length, 1), return_sequences=True))
  model.add(Dropout(dropout_rate))
  model.add(LSTM(lstm_neurons))
  model.add(Dropout(dropout_rate))

  # Add the Dense layer
  model.add(Dense(nb_chars, activation=dense_activation))

  return model

model = create_model()

Compile the model with the appropriate loss function and the adam optimizer

In [18]:
from keras.optimizers import Adam

def train_model(model):
  loss_function = 'categorical_crossentropy'

  optimizer = Adam()

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

train_model(model)

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

In [19]:
model.fit(X_train[:10], y_train[:10], epochs=20, batch_size=1)

Epoch 1/20
10/10 [==============================] - 4s 10ms/step - loss: 3.5889 - accuracy: 0.0000e+00
Epoch 2/20
10/10 [==============================] - 0s 8ms/step - loss: 3.4930 - accuracy: 0.3000
Epoch 3/20
10/10 [==============================] - 0s 7ms/step - loss: 3.2331 - accuracy: 0.3000
Epoch 4/20
10/10 [==============================] - 0s 9ms/step - loss: 2.3400 - accuracy: 0.3000
Epoch 5/20
10/10 [==============================] - 0s 7ms/step - loss: 2.0538 - accuracy: 0.3000
Epoch 6/20
10/10 [==============================] - 0s 8ms/step - loss: 1.9503 - accuracy: 0.3000
Epoch 7/20
10/10 [==============================] - 0s 8ms/step - loss: 1.8358 - accuracy: 0.5000
Epoch 8/20
10/10 [==============================] - 0s 10ms/step - loss: 1.7316 - accuracy: 0.3000
Epoch 9/20
10/10 [==============================] - 0s 7ms/step - loss: 1.8027 - accuracy: 0.3000
Epoch 10/20
10/10 [==============================] - 0s 8ms/step - loss: 1.9841 - accuracy: 0.2000
Epoch 11/20
1

If it does not overfit try to fix data prep and model architecture so it does

It overfit

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [20]:
# Create a function make_prediction(seed_start) wich takes a starting sequence and uses it to generate a molecule

def make_prediction(seed_start):
    seed = [dict_char_to_int[i] for i in seed_start]
    seed = np.array(seed)
    seed = seed[-seq_length:]
    seed = seed.reshape(1, seq_length, 1)
    seed = seed / len(unic_chars)
    
    prediction = model.predict(seed)
    
    prediction = np.argmax(prediction)
    
    return dict_int_to_char[prediction]

generate a molecule of your overfitted model

In [21]:
# Test it with a random seed
print(make_prediction("c1ccc2occ"))

1/1 [==============================] - 2s 2s/step
c


Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

In [22]:
"""from keras.callbacks import ModelCheckpoint

checkpoint_filepath = './checkpoint'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.load_weights(checkpoint_filepath)"""

"from keras.callbacks import ModelCheckpoint\n\ncheckpoint_filepath = './checkpoint'\nmodel_checkpoint_callback = ModelCheckpoint(\n    filepath=checkpoint_filepath,\n    save_weights_only=True,\n    monitor='val_accuracy',\n    mode='max',\n    save_best_only=True)\n\nmodel.load_weights(checkpoint_filepath)"

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

In [23]:
model = create_model()
train_model(model)

model.fit(X_train, y_train, epochs=10, batch_size=256)

Epoch 1/10
975/975 [==============================] - 11s 6ms/step - loss: 1.6345 - accuracy: 0.4935
Epoch 2/10
975/975 [==============================] - 6s 6ms/step - loss: 0.9647 - accuracy: 0.6618
Epoch 3/10
975/975 [==============================] - 6s 6ms/step - loss: 0.8684 - accuracy: 0.6860
Epoch 4/10
975/975 [==============================] - 6s 6ms/step - loss: 0.8236 - accuracy: 0.6960
Epoch 5/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7976 - accuracy: 0.7031
Epoch 6/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7793 - accuracy: 0.7081
Epoch 7/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7653 - accuracy: 0.7105
Epoch 8/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7556 - accuracy: 0.7130
Epoch 9/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7471 - accuracy: 0.7152
Epoch 10/10
975/975 [==============================] - 6s 6ms/step - loss: 0.7392 - accuracy: 0.716

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

In [48]:
import random
number = random.randrange(100, 1000)

tmp = df[df.smiles.str.len() < 50]
tmp = tmp[tmp.smiles.str.len() >= 10]

tmp = tmp.sample(n=number)
lst = tmp["smiles"].tolist()

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

In [28]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 55.1 MB/s eta 0:00:00


In [49]:
from rdkit.Chem import QED
from rdkit import Chem

for mol in lst:
    m = Chem.MolFromSmiles(mol)
    print(QED.default(m))

0.8214218920140605
0.7498981151339819
0.8279139144574026
0.8449665448279098
0.7458123362908268
0.8438347215153985
0.6437888286148052
0.8999260183068416
0.5974721331066543
0.7427322331808208
0.833225296393198
0.8476209888400327
0.8862895121879492
0.5326237524664265
0.8658441054253962
0.7465582412858817
0.8764963218252316
0.8682316673809644
0.914142313713413
0.5138589761146378
0.7838000655402724
0.8071467134459452
0.7957150668751548
0.7811556507039221
0.8398690636440699
0.8377098942320547
0.8960831634592834
0.8293109146842489
0.6618252585940767
0.602593285757422
0.7761141877503199
0.8373655543839724
0.821252024624438
0.789574596306847
0.8313198727055069
0.7990798388708751
0.7123842435236832
0.8592310132982628
0.6912911460861505
0.6870698175174704
0.8320489566275651
0.7938164258109834
0.8586858007273773
0.717977724021633
0.8927496307204699
0.8625967701774439
0.7961505900669251
0.7802452429016505
0.9314134491221163
0.6959966541021715
0.8121574132957391
0.7755762730011522
0.7941243265629064

Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules. 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better